In [41]:
import math
from utils import *
import pandas as pd
import numpy  as np
from numpy.linalg import norm
import fitz
from fitz import Rect
from line_utils import *
import re

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose  import ColumnTransformer, make_column_selector, make_column_transformer

In [42]:
pdf_file = "test_pdfs/LC002ALP100EV_2024.pdf"
doc              = fitz.open(pdf_file)
page             = doc[3]
page_dict        = page.get_text("dict",sort=True)
blocks           = page_dict["blocks"]
block            = blocks[6]
lines            = block['lines']

In [43]:
print(get_block_text(block))

TEXT 2 is an edited extract from the opening of Paul Murray’s novel,  The Bee Sting,  shortlisted 
for the 2023 Booker Prize.  The novel tells the tragi-comic story of the Barnes family, set in 
contemporary Ireland.  In this extract we meet the teenage daughter, Cass, and her best friend, 
Elaine. 
Cass and Elaine first met in Chemistry class, 
when Elaine poured iodine on Cass’s eczema 
during an experiment.  It was an accident; 
she’d cried more than Cass did, and insisted on 
going with her to the nurse.  They’d been 
friends ever since.  Every morning Cass called 
to Elaine’s house and they walked to school 
together.  At lunchtime, they rolled up their 
long skirts and wandered around the 
supermarket, listening to music from Elaine’s 
phone, eating croissants from the bakery 
section that were gone by the time they got to 
the checkout.   


In [44]:
print(lines[0].keys()) 
print(len(lines))
def line_is_empty(line):
    return all( [span["text"].isspace() for span in line["spans"]] )
lines = [line for line in lines if not line_is_empty(line)]
print(len(lines))

dict_keys(['spans', 'wmode', 'dir', 'bbox'])
19
17


In [45]:
print_line_table(lines)

x0       x1       y0       y1       dx       dy       fonts                                beginning                
------------------------------------------------------------------------------------------------------------------------
47.94    519.13   65.34    77.34    471.19   12.00    Calibri,Bold Calibri,BoldItalic      TEXT 2 is an edited extra
47.94    499.81   79.98    91.98    451.87   12.00    Calibri,Bold                         for the 2023 Booker Prize
47.94    525.21   94.62    106.62   477.27   12.00    Calibri,Bold                         contemporary Ireland.  In
47.94    84.01    109.26   121.26   36.07    12.00    Calibri,Bold                         Elaine.                  
47.94    262.08   138.54   150.54   214.14   12.00    Calibri                              Cass and Elaine first met
47.94    268.27   153.24   165.24   220.33   12.00    Calibri                              when Elaine poured iodine
47.94    254.96   167.88   179.88   207.02   12.00    Calibr

In [46]:
pd.set_option("display.float_format", "{:.2f}".format)
df = get_line_df(lines)
df.head(10)

,x0,y0,x1,y1,dL,n_spans,font_list,common_font,mode_font,n_words,w,h,text
0,47.94,65.34,519.13,77.34,14.64,3,"[Calibri,Bold, Calibri,BoldItalic, Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,471.19,12.00,TEXT 2 is an edited extract from the opening o...
1,47.94,79.98,499.81,91.98,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,451.87,12.00,for the 2023 Booker Prize. The novel tells th...
2,47.94,94.62,525.21,106.62,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",15,477.27,12.00,contemporary Ireland. In this extract we meet...
3,47.94,109.26,84.01,121.26,29.28,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",1,36.07,12.00,Elaine.
4,47.94,138.54,262.08,150.54,14.70,1,[Calibri],Calibri,Calibri,8,214.14,12.00,"Cass and Elaine first met in Chemistry class,"
5,47.94,153.24,268.27,165.24,14.64,1,[Calibri],Calibri,Calibri,8,220.33,12.00,when Elaine poured iodine on Cass’s eczema
6,47.94,167.88,254.96,179.88,14.64,1,[Calibri],Calibri,Calibri,7,207.02,12.00,during an experiment. It was an accident;
7,47.94,182.52,279.52,194.52,14.64,1,[Calibri],Calibri,Calibri,10,231.58,12.00,"she’d cried more than Cass did, and insisted on"
8,47.94,197.16,251.64,209.16,14.64,1,[Calibri],Calibri,Calibri,9,203.70,12.00,going with her to the nurse. They’d been
9,47.94,211.80,270.93,223.80,14.64,1,[Calibri],Calibri,Calibri,7,222.99,12.00,friends ever since. Every morning Cass called


# Preprocessing dataframe

In [47]:
X = df.drop(columns=["font_list","text","n_spans","dL","n_words"])

num_vars = list( X.select_dtypes(include=np.number).columns )
cat_vars = list( X.select_dtypes(include='object').columns  )

X[num_vars] = StandardScaler().fit_transform(X[num_vars])

ohe = OneHotEncoder(drop="if_binary", sparse_output=False, handle_unknown="error" )
X[cat_vars] = ohe.fit_transform(X[cat_vars])

basic_preproc = make_column_transformer(
    (StandardScaler(), num_vars),
    (OneHotEncoder(drop="if_binary",sparse_output=False, handle_unknown="error"), cat_vars),
    remainder="drop"
    )
basic_kmeans = make_pipeline(basic_preproc, KMeans(n_clusters=2,  n_init=400))


display(X.head(2))
display(pd.DataFrame(basic_preproc.fit_transform(df),columns=num_vars+ cat_vars).head(2))

,x0,y0,x1,y1,common_font,mode_font,w,h
0,0.00,-1.68,1.97,-1.68,1.00,1.00,1.97,0.00
1,0.00,-1.49,1.81,-1.49,1.00,1.00,1.81,0.00


,x0,y0,x1,y1,w,h,common_font,mode_font
0,0.00,-1.68,1.97,-1.68,1.97,0.00,1.00,1.00
1,0.00,-1.49,1.81,-1.49,1.81,0.00,1.00,1.00


# Clustering

## Default Kmeans

In [48]:
display_df = df.copy()
display_df["cluster"] = basic_kmeans.fit_predict(df)
display_df.head(7)

,x0,y0,x1,y1,dL,n_spans,font_list,common_font,mode_font,n_words,w,h,text,cluster
0,47.94,65.34,519.13,77.34,14.64,3,"[Calibri,Bold, Calibri,BoldItalic, Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,471.19,12.00,TEXT 2 is an edited extract from the opening o...,1
1,47.94,79.98,499.81,91.98,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,451.87,12.00,for the 2023 Booker Prize. The novel tells th...,1
2,47.94,94.62,525.21,106.62,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",15,477.27,12.00,contemporary Ireland. In this extract we meet...,1
3,47.94,109.26,84.01,121.26,29.28,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",1,36.07,12.00,Elaine.,0
4,47.94,138.54,262.08,150.54,14.70,1,[Calibri],Calibri,Calibri,8,214.14,12.00,"Cass and Elaine first met in Chemistry class,",0
5,47.94,153.24,268.27,165.24,14.64,1,[Calibri],Calibri,Calibri,8,220.33,12.00,when Elaine poured iodine on Cass’s eczema,0
6,47.94,167.88,254.96,179.88,14.64,1,[Calibri],Calibri,Calibri,7,207.02,12.00,during an experiment. It was an accident;,0


In [49]:
top_init    = X.iloc[0]
bottom_init = X.iloc[-1]  
init_centroids = [ X.iloc[0], X.iloc[-1] ]

kmeans = KMeans(n_clusters=2, random_state=42,init=init_centroids, n_init="auto")
y_pred = kmeans.fit_predict(X)
y_pred

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## Weighted Kmeans

In [50]:
X_weighted = X.copy()

y_weight    = math.sqrt(2)
font_weight = math.sqrt(4)

X_weighted[["y0","y1"]]                 = X[["y0","y1"]]*y_weight
X_weighted[["common_font","mode_font"]] = X[["common_font","mode_font"]]*font_weight 
X_weighted.head()

,x0,y0,x1,y1,common_font,mode_font,w,h
0,0.00,-2.37,1.97,-2.37,2.00,2.00,1.97,0.00
1,0.00,-2.10,1.81,-2.10,2.00,2.00,1.81,0.00
2,0.00,-1.83,2.02,-1.83,2.00,2.00,2.02,0.00
3,0.00,-1.56,-1.74,-1.56,2.00,2.00,-1.74,0.00
4,0.00,-1.02,-0.22,-1.02,0.00,0.00,-0.22,0.00


In [51]:
init_centroids = [ X.iloc[0], X.iloc[-1] ]
kmeans = KMeans(n_clusters=2, random_state=42,init=init_centroids, n_init="auto")
#kmeans = KMeans(n_clusters=2, n_init=1000)
cluster_pred = kmeans.fit_predict(X_weighted)
pd.concat((X_weighted,pd.Series(cluster_pred,name="Cluster") ),axis=1).head(7)

,x0,y0,x1,y1,common_font,mode_font,w,h,Cluster
0,0.00,-2.37,1.97,-2.37,2.00,2.00,1.97,0.00,0
1,0.00,-2.10,1.81,-2.10,2.00,2.00,1.81,0.00,0
2,0.00,-1.83,2.02,-1.83,2.00,2.00,2.02,0.00,0
3,0.00,-1.56,-1.74,-1.56,2.00,2.00,-1.74,0.00,0
4,0.00,-1.02,-0.22,-1.02,0.00,0.00,-0.22,0.00,1
5,0.00,-0.75,-0.17,-0.75,0.00,0.00,-0.17,0.00,1
6,0.00,-0.48,-0.28,-0.48,0.00,0.00,-0.28,0.00,1


# Full custom K-means

## Pre proc X

In [52]:
X = basic_preproc.fit_transform(df)
X.shape

(17, 8)

In [53]:
X_df = pd.DataFrame(basic_preproc.fit_transform(df),columns=num_vars+ cat_vars)
X_df.head(6)

,x0,y0,x1,y1,w,h,common_font,mode_font
0,0.00,-1.68,1.97,-1.68,1.97,0.00,1.00,1.00
1,0.00,-1.49,1.81,-1.49,1.81,0.00,1.00,1.00
2,0.00,-1.30,2.02,-1.30,2.02,0.00,1.00,1.00
3,0.00,-1.10,-1.74,-1.10,-1.74,0.00,1.00,1.00
4,0.00,-0.72,-0.22,-0.72,-0.22,0.00,0.00,0.00
5,0.00,-0.53,-0.17,-0.53,-0.17,0.00,0.00,0.00


## Initialise clusters

In [54]:
clusts = X[[0, X.shape[0]-1]]
clust0, clust1 = clusts
clusts.shape

(2, 8)

## Calculate cluster distances

In [55]:
dist0 = (clust0-X[0]).T@(clust0-X[0])
dist1 = (clust1-X[0]).T@(clust1-X[0])
print(np.sqrt(dist0),np.sqrt(dist1))
dists = [norm( clust - X[0]) for clust in clusts]
print(dists[0],dists[1])

0.0 6.788481725207638
0.0 6.788481725207638


In [56]:
print(X.shape)
print(clusts.shape)
dist0 = norm(X-clusts[0],axis=1)
dist1 = norm(X-clusts[1],axis=1)
dists = np.vstack((dist0, dist1)).T
print(dists.shape)

(17, 8)
(2, 8)
(17, 2)


### Fully vectorised

In [57]:
diff = X[:, np.newaxis, :] - clusts[np.newaxis, :, :]  #  (17, 2, 8)
dists = np.linalg.norm(diff, axis=2)  #  (17, 2)

### Examine distance components for edge point

In [58]:
print(f"{'i':<5} {'clust':<5} {'l':<8} {'dy0':8} {'dx1':8} {'dw':8} {'dfont':8}")
for i, x in enumerate(X):
    for j, clust in enumerate(clusts):
        l = norm(x - clust)
        dr = (x - clust)**2
        dw    = dr[4]
        dy0   = dr[3]
        dx1   = dr[2]
        dfont = dr[6]
        if i == 3:  
            print(f"{i:<5} {j:<5} {l:<8.2f} {dy0:<8.2f} {dx1:<8.2f} {dw:<8.2f} {dfont:<8.2f}")

i     clust l        dy0      dx1      dw       dfont   
3     0     5.31     0.33     13.77    13.77    0.00    
3     1     4.08     7.20     0.11     0.11     1.00    


## Label data points

In [59]:
y_bool = dists[:,0]< dists[:,1]
y = np.array( y_bool ,dtype= np.int64 )

print("Cluster 0\nShape:",X[y_bool].shape)
print("Cluster 1\nShape:",X[~y_bool].shape)

X_df_labelled = pd.concat((X_df,pd.Series(y,name="cluster")), axis=1) 
X_df_labelled.head(5)


Cluster 0
Shape: (3, 8)
Cluster 1
Shape: (14, 8)


,x0,y0,x1,y1,w,h,common_font,mode_font,cluster
0,0.00,-1.68,1.97,-1.68,1.97,0.00,1.00,1.00,1
1,0.00,-1.49,1.81,-1.49,1.81,0.00,1.00,1.00,1
2,0.00,-1.30,2.02,-1.30,2.02,0.00,1.00,1.00,1
3,0.00,-1.10,-1.74,-1.10,-1.74,0.00,1.00,1.00,0
4,0.00,-0.72,-0.22,-0.72,-0.22,0.00,0.00,0.00,0


### Fully vectorised

In [60]:
labels = np.argmin(dists, axis=1)  # shape (17,) 
k = clusts.shape[0]  # number of clusters (e.g. 2)

# Use list comprehension to compute new means per cluster label
new_clusts = np.vstack([X[labels == i].mean(axis=0) for i in range(k)])

## Recalculate cluster centres

In [61]:
clust0 = np.mean(X[y_bool], axis=0)
clust1 = np.mean(X[~y_bool], axis=0 )
new_clusts = np.vstack( (clust0,clust1))

In [62]:
print(X.shape)
print(new_clusts.shape)
dist0 = np.linalg.norm(X-new_clusts[0],axis=1)
dist1 = np.linalg.norm(X-new_clusts[1],axis=1)

dists = np.vstack((dist0, dist1)).T
dists.shape

(17, 8)
(2, 8)


(17, 2)

In [63]:
print(X[:, np.newaxis, :].shape)
print(clusts[np.newaxis, :, :].shape)
diff = X[:, np.newaxis, :] - clusts[np.newaxis, :, :]  #  (17, 2, 8)
dists = np.linalg.norm(diff, axis=2)  #  (17, 2)

(17, 1, 8)
(1, 2, 8)


## Check cluster displacement

In [64]:
dclust = new_clusts - clusts
print(dclust.shape)
clust_delta = norm(dclust, axis=1)

(2, 8)


In [65]:
# Relabel according to new_clust
if clust_delta[0] < tol and clust_delta[1] < tol:
    break
else:
    # recalculate clusts
    # relabel points

SyntaxError: incomplete input (762121560.py, line 6)

In [ ]:
print(X.shape)
i_nword = X.shape[1]-1

In [ ]:
full_vect = X[:,:i_nword]
full_vect.shape

## One Iteration Custom Cluster

### Define dataframe and word mask

In [ ]:
df        = get_line_df(lines)

# We need to choose now the rows where the number of words is below 4
word_mask = df["n_words"].to_numpy() < 4

print("Raw lines dataframe:")
display(df.head(10))

Raw lines dataframe:


,x0,y0,x1,y1,dL,n_spans,font_list,common_font,mode_font,n_words,w,h,text
0,47.94,65.34,519.13,77.34,14.64,3,"[Calibri,Bold, Calibri,BoldItalic, Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,471.19,12.00,TEXT 2 is an edited extract from the opening o...
1,47.94,79.98,499.81,91.98,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",18,451.87,12.00,for the 2023 Booker Prize. The novel tells th...
2,47.94,94.62,525.21,106.62,14.64,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",15,477.27,12.00,contemporary Ireland. In this extract we meet...
3,47.94,109.26,84.01,121.26,29.28,1,"[Calibri,Bold]","Calibri,Bold","Calibri,Bold",1,36.07,12.00,Elaine.
4,47.94,138.54,262.08,150.54,14.70,1,[Calibri],Calibri,Calibri,8,214.14,12.00,"Cass and Elaine first met in Chemistry class,"
5,47.94,153.24,268.27,165.24,14.64,1,[Calibri],Calibri,Calibri,8,220.33,12.00,when Elaine poured iodine on Cass’s eczema
6,47.94,167.88,254.96,179.88,14.64,1,[Calibri],Calibri,Calibri,7,207.02,12.00,during an experiment. It was an accident;
7,47.94,182.52,279.52,194.52,14.64,1,[Calibri],Calibri,Calibri,10,231.58,12.00,"she’d cried more than Cass did, and insisted on"
8,47.94,197.16,251.64,209.16,14.64,1,[Calibri],Calibri,Calibri,9,203.70,12.00,going with her to the nurse. They’d been
9,47.94,211.80,270.93,223.80,14.64,1,[Calibri],Calibri,Calibri,7,222.99,12.00,friends ever since. Every morning Cass called


## Preprocess data frame

In [ ]:
# These cols of the df are not informative for text-block clustering.
bad_nums = ["n_spans","dL","x1","n_words"]
bad_cats = ["font_list","text"]

num_vars = [ col for col in  df.select_dtypes(include=np.number).columns if col not in bad_nums] 
cat_vars = [ col for col in  df.select_dtypes(include='object').columns  if col not in bad_cats] 

basic_preproc = make_column_transformer(
    (StandardScaler(), num_vars),
    (OneHotEncoder(drop="if_binary",sparse_output=False, handle_unknown="error"), cat_vars),
    remainder="drop"
    )
X_cols = num_vars + cat_vars 
X      = basic_preproc.fit_transform(df)
X_df   = pd.DataFrame(X,columns=X_cols )
print(f"Preprocessed dataframe of shape {X.shape}:")
print(X_df.head(8),"\n")

Preprocessed dataframe of shape (17, 7):
    x0    y0    y1     w    h  common_font  mode_font
0 0.00 -1.68 -1.68  1.97 0.00         1.00       1.00
1 0.00 -1.49 -1.49  1.81 0.00         1.00       1.00
2 0.00 -1.30 -1.30  2.02 0.00         1.00       1.00
3 0.00 -1.10 -1.10 -1.74 0.00         1.00       1.00
4 0.00 -0.72 -0.72 -0.22 0.00         0.00       0.00
5 0.00 -0.53 -0.53 -0.17 0.00         0.00       0.00
6 0.00 -0.34 -0.34 -0.28 0.00         0.00       0.00
7 0.00 -0.15 -0.15 -0.07 0.00         0.00       0.00 



## Initialise clusters

In [ ]:
# initialise clusters - first and last data point are top and bottom of page
k=2
m, n = X.shape
clusts  = X[[0, m-1]]
d_clust = norm(clusts,axis=1)
inertia = d_clust.T@d_clust
i_w       = X_cols.index("w")
print(clusts.shape)
print(d_clust , inertia)

(2, 7)
[3.39549044 2.63902596] 18.493813338810476


## Normal distance calc

In [ ]:
# full distance calc for certain, N-1 dimensional for others.
full_vect  = X[~word_mask, :]
full_clust = clusts[:, :]

full_diff   = full_vect[:, np.newaxis, :] - full_clust[np.newaxis, :, :]  #  (m_full, 2, n)
full_dists  = norm(full_diff, axis=2)                                     #  (m_full, 2)

print(f"Full vector of shape {full_vect.shape}")
print(pd.DataFrame(full_vect, columns= X_cols).head(8),"\n\n" )


Full vector of shape (15, 7)
    x0    y0    y1     w    h  common_font  mode_font
0 0.00 -1.68 -1.68  1.97 0.00         1.00       1.00
1 0.00 -1.49 -1.49  1.81 0.00         1.00       1.00
2 0.00 -1.30 -1.30  2.02 0.00         1.00       1.00
3 0.00 -0.72 -0.72 -0.22 0.00         0.00       0.00
4 0.00 -0.53 -0.53 -0.17 0.00         0.00       0.00
5 0.00 -0.34 -0.34 -0.28 0.00         0.00       0.00
6 0.00 -0.15 -0.15 -0.07 0.00         0.00       0.00
7 0.00  0.05  0.05 -0.31 0.00         0.00       0.00 




## Distance for few-word lines

In [ ]:
# If we have a line with a small n_words, the width is no longer a good variable for clustering.
small_vect  = np.delete(X[word_mask], i_w, axis=1)
small_clust = np.delete(clusts,       i_w, axis=1)

small_diff   = small_vect[:, np.newaxis, :] - small_clust[np.newaxis, :, :]  #  (m_small, 2, n -1)
small_dists  = norm(small_diff, axis=2)                                      #  (m_small, 2)

small_cols = [i for i in X_cols if i != "w" ]

print(f"Width-excluded vector of shape {small_vect.shape}")
print(pd.DataFrame(small_vect, columns = small_cols).head(2),"\n\n")

Width-excluded vector of shape (2, 6)
    x0    y0    y1    h  common_font  mode_font
0 0.00 -1.10 -1.10 0.00         1.00       1.00
1 0.00  1.58  1.58 0.00         0.00       0.00 




## Combine distances  - label points

In [ ]:
# Combine distances and label 
dists = np.empty((m, k))
dists[word_mask]  = small_dists
dists[~word_mask] = full_dists
labels = np.argmin(dists, axis=1)

X_df["cluster"] = pd.Series(labels)
X_df.head(8)

,x0,y0,y1,w,h,common_font,mode_font,cluster
0,0.00,-1.68,-1.68,1.97,0.00,1.00,1.00,0
1,0.00,-1.49,-1.49,1.81,0.00,1.00,1.00,0
2,0.00,-1.30,-1.30,2.02,0.00,1.00,1.00,0
3,0.00,-1.10,-1.10,-1.74,0.00,1.00,1.00,0
4,0.00,-0.72,-0.72,-0.22,0.00,0.00,0.00,0
5,0.00,-0.53,-0.53,-0.17,0.00,0.00,0.00,0
6,0.00,-0.34,-0.34,-0.28,0.00,0.00,0.00,1
7,0.00,-0.15,-0.15,-0.07,0.00,0.00,0.00,1


## Calculate new clusters

In [ ]:
new_clusts = np.vstack([X[labels == i].mean(axis=0) for i in range(k)])

norm_change =  norm(clusts-new_clusts,axis=1)
norm_clust  =  norm(clusts,axis = 1)

tol = 0.01
if all(norm_change/norm_clust) < tol:
    break

In [ ]:
tol = 0.01
all(norm_change/norm_clust) < tol 


False